In [ ]:
#Script to pull out most advantageous options from a csv file.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as si
from numba import njit, prange

# Load option data (assumes 1,000,000 options)
# Columns: ["Type", "S", "K", "T", "r", "sigma", "Premium"]
df = pd.read_csv("large_options_data.csv")

# Black-Scholes Formula for European Call/Put Pricing
@njit(parallel=True)
def black_scholes_option_value(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    call_price = S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    put_price = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * si.norm.cdf(-d1)
    
    return np.where(option_type == "call", call_price, put_price)

# Compute Intrinsic Value, Expected Profit, and Rank
@njit(parallel=True)
def process_options(S, K, T, r, sigma, premium, option_type):
    option_values = black_scholes_option_value(S, K, T, r, sigma, option_type)
    intrinsic_values = np.where(option_type == "call", np.maximum(S - K, 0), np.maximum(K - S, 0))
    expected_profits = intrinsic_values - premium  # Profit if exercised

    return option_values, intrinsic_values, expected_profits

# Convert DataFrame to NumPy arrays for faster processing
S = df["S"].values
K = df["K"].values
T = df["T"].values
r = df["r"].values
sigma = df["sigma"].values
premium = df["Premium"].values
option_type = df["Type"].values  # "call" or "put"

# Process options in parallel
option_values, intrinsic_values, expected_profits = process_options(S, K, T, r, sigma, premium, option_type)

# Add results to DataFrame
df["Intrinsic_Value"] = intrinsic_values
df["Expected_Profit"] = expected_profits

# Filter only profitable (ITM) options
df_itm = df[df["Expected_Profit"] > 0]

# Rank options by highest expected profit
df_itm_sorted = df_itm.sort_values(by="Expected_Profit", ascending=False)

# Save top profitable options
df_itm_sorted.to_csv("most_advantageous_options.csv", index=False)

print(f"Filtered & ranked options saved to 'most_advantageous_options.csv'.")
